# Financial Forecasting Model Training Notebook

This notebook demonstrates how to build a financial forecasting model using historical price data. We create lag features and a moving average, then train a RandomForestRegressor to predict future closing prices.

In [ ]:
# Import necessary libraries for data manipulation and modeling
import pandas as pd  # For data manipulation
import numpy as np   # For numerical operations
from sklearn.model_selection import train_test_split  # For splitting the data
from sklearn.ensemble import RandomForestRegressor   # Regression model
from sklearn.metrics import mean_squared_error       # For model evaluation
import matplotlib.pyplot as plt  # For plotting
import pickle  # For saving the model

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# Load historical price data
# Ensure that the CSV file is located in the '../data/' folder
data = pd.read_csv('../data/historical_prices.csv', parse_dates=['Date'])
data.sort_values('Date', inplace=True)  # Sort by date

# Display the first few rows to verify the data
print(data.head())

In [ ]:
# Feature Engineering: Create lag features and moving average
# Create a lag feature for the previous day's closing price
data['Close_Lag1'] = data['Close'].shift(1)

# Create a 5-day moving average of the closing price
data['MA_5'] = data['Close'].rolling(window=5).mean()

# Remove rows with NaN values created by shifting and rolling
data.dropna(inplace=True)

# Verify the new features
print(data.head())

In [ ]:
# Define features and target variable
features = ['Close_Lag1', 'MA_5']  # Predictors
target = 'Close'                  # What we want to forecast

X = data[features]
y = data[target]

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Display the shapes of the datasets
print('Training set shape:', X_train.shape)
print('Testing set shape:', X_test.shape)

In [ ]:
# Train a RandomForestRegressor model on the training data
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
predictions = model.predict(X_test)

# Evaluate the model using Root Mean Squared Error (RMSE)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print('Test RMSE:', rmse)

In [ ]:
# Plot the actual vs. predicted closing prices for visualization
plt.figure(figsize=(10, 6))
plt.plot(data['Date'].iloc[X_train.shape[0]:], y_test, label='Actual Close')
plt.plot(data['Date'].iloc[X_train.shape[0]:], predictions, label='Predicted Close')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Actual vs Predicted Closing Prices')
plt.legend()
plt.show()

In [ ]:
# Save the trained model to the '../model/' directory
model_save_path = '../model/financial_forecasting_model.pkl'
with open(model_save_path, 'wb') as f:
    pickle.dump(model, f)

print('Trained model saved successfully!')

### Next Steps

Once the model is trained, you can use it to forecast future prices. Consider integrating the model into a web app or API endpoint for real-time predictions.